In [1]:
import sqlite3
import pandas as pd
from pathlib import Path

In [2]:
Path("database.db").unlink(missing_ok=True)

In [3]:
conn = sqlite3.connect('database.db')
cursor = conn.cursor()

# Table creation and data ingestion

In [4]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS conversations (
    id INTEGER PRIMARY KEY,
    text TEXT
)
''')
conn.commit()

In [5]:
df = pd.read_csv("data/data-v1.csv")
print(len(df))
df.head()

2347


,id,text,tag
0,0,"Agent: Hi, this is Kevin from Holland and Barr...",store closing hours
1,1,"Agent: Good morning, Holland and Barrett custo...",repeat purchases
2,2,"Agent: Hi, thank you for contacting Holland an...",supplement advice
3,3,"Agent: Hello, Holland & Barrett customer suppo...",staff interaction
4,4,Agent: Thank you for calling Holland & Barrett...,hair products


In [6]:
df_conversations = df[["id", "text"]]
print(len(df_conversations))
df_conversations.head()

2347


,id,text
0,0,"Agent: Hi, this is Kevin from Holland and Barr..."
1,1,"Agent: Good morning, Holland and Barrett custo..."
2,2,"Agent: Hi, thank you for contacting Holland an..."
3,3,"Agent: Hello, Holland & Barrett customer suppo..."
4,4,Agent: Thank you for calling Holland & Barrett...


In [7]:
df_conversations.to_sql('conversations', conn, if_exists='append', index=False)

2347

In [16]:
cursor.execute('DROP TABLE IF EXISTS features')
conn.commit()

In [17]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS features (
    conversation_id INTEGER,
    tag TEXT,
    user_final_sentiment TEXT,
    FOREIGN KEY (conversation_id) REFERENCES conversations(id)
)
''')
conn.commit()

In [23]:
df_sentiment = pd.read_csv("data/sentiment.csv")
print(len(df_sentiment))
df_sentiment.head()

2347


,conversation_id,user_final_sentiment
0,0,Positive
1,1,Positive
2,2,Positive
3,3,Positive
4,4,Positive


In [24]:
df_tag = pd.read_csv("data/tag-updated.csv")
print(len(df_tag))
df_tag.head()

2347


,conversation_id,tag
0,0,Store Early Closure
1,1,repeat purchases
2,2,Stress & Anxiety Supplements
3,3,Store Staff Service
4,4,hair products


In [25]:
df_features = pd.merge(df_sentiment, df_tag, on="conversation_id")
print(len(df_features))
df_features.head()

2347


,conversation_id,user_final_sentiment,tag
0,0,Positive,Store Early Closure
1,1,Positive,repeat purchases
2,2,Positive,Stress & Anxiety Supplements
3,3,Positive,Store Staff Service
4,4,Positive,hair products


In [26]:
df_features.to_sql("features", conn, if_exists='append', index=False)

2347

In [27]:
df_distilled = pd.read_csv("data/distilled.csv")
print(len(df_distilled))
df_distilled.head()

2347


,conversation_id,key_issue,key_resolution
0,0,Store unexpectedly closed.,Store closure due to power outage.
1,1,Missing order confirmation email.,Order confirmed; email confirmation missing du...
2,2,Seeking supplement advice for anxiety.,Supplement recommendations for anxiety: 5-HTP ...
3,3,Rude and unhelpful staff.,Feedback on rude staff relayed to store manager.
4,4,Vegetarian suitability of hair loss capsules.,Confirmed vegetarian suitability of product.


In [28]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS distilled (
    conversation_id INTEGER,
    key_issue TEXT,
    key_resolution TEXT,
    FOREIGN KEY (conversation_id) REFERENCES conversation(id)
)
''')
conn.commit()

In [29]:
df_distilled.to_sql("distilled", conn, if_exists='append', index=False)

2347